# Setup

Remember to install CUDA dependencies

In [1]:
import glob
import os
from ultralytics import YOLO
import random
import shutil
import re

In [2]:
model = YOLO('yolo11n.yaml', task='detect')  # build a new model from YAML

# Sorting Kaggle dataset into train/val

Source: https://www.kaggle.com/datasets/hamidl/yoloqrlabeled?resource=download

In [3]:
# create required directories
if not os.path.exists('../data/kaggle'):
    os.mkdir('../data/kaggle')
    os.mkdir('../data/kaggle/images')
    os.mkdir('../data/kaggle/images/train')
    os.mkdir('../data/kaggle/images/val')
    os.mkdir('../data/kaggle/labels')
    os.mkdir('../data/kaggle/labels/train')
    os.mkdir('../data/kaggle/labels/val')

# copy images and labels to required directories by train/val splits

# total images
total = len(glob.glob('../data/YOLO-QR-datasets/Dataset 1/images/*.jpg')) + \
        len(glob.glob('../data/YOLO-QR-datasets/Dataset 2/images/*.jpg')) + \
        len(glob.glob('../data/YOLO-QR-datasets/Dataset 3/images/*.jpg'))

train = int(total * 0.9) # floored
val = int(total * 0.1) # floored
print(f'Train : {train}')
print(f'Val   : {val}')

# getting filepaths of all images and annotations
filepaths = {'images': [], 'labels': []}
for i in range(1, 4):
    # images
    for file in glob.glob(f'../data/YOLO-QR-datasets/Dataset {i}/images/*.jpg'):
        filepaths['images'].append(file)
    # labels
    for file in glob.glob(f'../data/YOLO-QR-datasets/Dataset {i}/labels/*.txt'):
        filepaths['labels'].append(file)

print(filepaths['images'][:2])
print(filepaths['labels'][:2])

# shuffled indexes
indexes = list(range(total))
random.shuffle(indexes)
print(indexes[:2])

Train : 18310
Val   : 2034
['../data/YOLO-QR-datasets/Dataset 1/images\\1.jpg', '../data/YOLO-QR-datasets/Dataset 1/images\\101.jpg']
['../data/YOLO-QR-datasets/Dataset 1/labels\\1.txt', '../data/YOLO-QR-datasets/Dataset 1/labels\\101.txt']
[16042, 833]


In [4]:
# copy images and labels to required directories with train/val splits
pattern = r'\d+' # regex pattern for grabbing first number in label file
for i, index in enumerate(indexes):
    if i < train:
        # train

        # copy image file as-is
        shutil.copyfile(filepaths['images'][index], f'../data/kaggle/images/train/{i}.jpg')

        # copy label file but change class index to 0
        with open(filepaths['labels'][index], 'r') as f:
            line = f.readline()
        num = re.search(pattern, line).group()
        line = line.replace(num, '0', 1)
        with open(f'../data/kaggle/labels/train/{i}.txt', 'w') as f:
            f.write(line)
    
    else:
        # val
        shutil.copyfile(filepaths['images'][index], f'../data/kaggle/images/val/{i}.jpg')
        with open(filepaths['labels'][index], 'r') as f:
            line = f.readline()
        num = re.search(pattern, line).group()
        line = line.replace(num, '0', 1)
        with open(f'../data/kaggle/labels/val/{i}.txt', 'w') as f:
            f.write(line)

In [5]:
# yaml file for training YOLO on
yaml = '''
path: ../data/kaggle
train: images/train
val: images/val

nc: 1
names: [\'QR\']
'''

with open('../data/kaggle/kaggle-QR.yaml', 'w') as f:
    f.write(yaml)

# Training

In [6]:
model.train(data='../data/kaggle/kaggle-QR.yaml', epochs=100, imgsz=640, single_cls=True)

New https://pypi.org/project/ultralytics/8.3.6 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.3  Python-3.12.6 torch-2.4.1 CPU (Intel Core(TM) i7-1065G7 1.30GHz)
engine\trainer: task=detect, mode=train, model=yolo11n.yaml, data=../data/kaggle/kaggle-QR.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=True, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_fr

train: Scanning C:\Users\aidan\OneDrive\Desktop\itu\msc\courses\sem3\research_project\Research-Project-Data-Matrix-Code-\data\kaggle\labels\train... 18310 images, 0 backgrounds, 0 corrupt: 100%|██████████| 18310/18310 [00:20<00:00, 879.31it/s]


train: New cache created: C:\Users\aidan\OneDrive\Desktop\itu\msc\courses\sem3\research_project\Research-Project-Data-Matrix-Code-\data\kaggle\labels\train.cache


val: Scanning C:\Users\aidan\OneDrive\Desktop\itu\msc\courses\sem3\research_project\Research-Project-Data-Matrix-Code-\data\kaggle\labels\val... 2035 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2035/2035 [00:02<00:00, 987.21it/s] 


val: New cache created: C:\Users\aidan\OneDrive\Desktop\itu\msc\courses\sem3\research_project\Research-Project-Data-Matrix-Code-\data\kaggle\labels\val.cache
Plotting labels to runs\detect\train\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs\detect\train
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/100         0G      2.982      3.511      4.327         46        640:   9%|▉         | 106/1145 [14:52<2:25:44,  8.42s/it]


KeyboardInterrupt: 